In [14]:
curve_u = []
curve_l = []
curves = {}
parameters = dict()

In = 'f.I0'
Isc = '1000*f.Isc'

tol_im_p = 10
tol_im_m = 10

tol_tm_p = 20
tol_tm_m = 20

tol_m_i2t_p = 20
tol_m_i2t_m = 20

i_i2t = 12

select_expr_list = ['d.trip_on is True and d.i2t_on is False',
                    'd.trip_on is True and d.i2t_on is True',
                    'd.trip_on is False']

def get_curves(Ir, I1, T_conv, Im, Tm, i2t_tol):
    k_I2T = str(i_i2t**2) + '*d.t_m'
    # M
    curve1 = [('point', I1, T_conv),
                ('point', I1, Tm),
                ('point', Isc, Tm)]
    # M, I2t
    curve2 = [('point', I1, T_conv),
                ('I2T', i2t_tol, Ir, Im, Isc, Tm, 100, k_I2T, 2),
                ('point', Isc, Tm)]
    return curve1, curve2

# Upper curves
Ir = In
T_conv = '3600 if f.In <= 63  else 2*3600'
Im = 'd.i_m*' + Ir + '*' + str((100+tol_im_p)/100)
I1 = Im
Tm = 'd.t_m*' + str((100+tol_tm_p)/100)
i2t_tol = (100+tol_m_i2t_p)/100
curve_u1, curve_u2 = get_curves(Ir, I1, T_conv, Im, Tm, i2t_tol)
# Lower curves
Ir = In
T_conv = '3600 if f.In <= 63  else 2*3600'
Im = 'd.i_m*' + Ir + '*' + str((100-tol_im_m)/100)
I1 = Im
Tm = 'd.t_m*' + str((100-tol_tm_m)/100)
i2t_tol = (100-tol_m_i2t_m)/100
curve_l1, curve_l2 = get_curves(Ir, I1, T_conv, Im, Tm, i2t_tol)

parameters = {  "trip_on" : ["Trip unit enabled ?", "", False, None, "Whether trip unit is enabled", "bool"],
                'i_m'       : ['Ig', 'xI0', 0.2, [0.2,0.3,0.4,0.5,0.6,0.7,0.8,1], 'Ground fault pickup current'],
                't_m'       : ['tg', 's', 0.1, [0.1,0.2,0.5,1], 'Short-time tripping delay (@ 10xIg for I2T=ON)'],
                'i2t_on'    : ['I2t ON', '', False, None, '', 'bool'],
            }

curves = {'select_expr_list': select_expr_list,
          'curve_u1': curve_u1, 'curve_l1': curve_l1,
         'curve_u2': curve_u2, 'curve_l2': curve_l2,
         'curve_u3': [], 'curve_l3': []}


In [15]:
import json, re

data = {'type'          : 'protection',
        'parameters'    : parameters,
        'data'          : curves,
        'graph_model'   : []}

with open('legrand_mp4_g.json', 'w') as fp:
    json.dump(data, fp, indent=2)

# output = json.dumps(data, indent=2)
# output2 = re.sub(r'": \[\s+', '": [', output)
# output3 = re.sub(r'",\s+', '", ', output2)
# output4 = re.sub(r'"\s+\]', '"]', output3)

# with open('legrand_uP.json', 'w') as fp:
#     fp.write(output4)